In [ ]:
# Imports
import csv
import json
import numpy as np
import os

In [ ]:
# String constants
SCENARIO_ROOT = './scenarios/'
BOTTLENECK_OSM = SCENARIO_ROOT + 'bottleneck_osm.scenario'
BOTTLENECK_GNM = SCENARIO_ROOT + 'bottleneck_gnm.scenario'

In [ ]:
# Running scenarios
# !!! vadere-console.jar must be placed in the same folder as this notebook!!!
def run_scenario(scenario_name):
    java_command = 'java -jar vadere-console.jar scenario-run'
    command = java_command + ' --scenario-file "scenarios/' + scenario_name + '.scenario" --output-dir="output"'
    os.system(command)

In [ ]:
def addPedestrian(in_file, out_file, ped_id, x, y, target_ids, speed):
    ped = {
      "source" : None,
      "targetIds" : target_ids,
      "position" : {
        "x" : x,
        "y" : y
      },
      "velocity" : {
        "x" : 0.0,
        "y" : 0.0
      },
      "nextTargetListIndex" : 0,
      "freeFlowSpeed" : speed,
      "attributes" : {
        "id" : ped_id,
        "radius" : 0.2,
        "densityDependentSpeed" : False,
        "speedDistributionMean" : 1.34,
        "speedDistributionStandardDeviation" : 0.26,
        "minimumSpeed" : 0.5,
        "maximumSpeed" : 2.2,
        "acceleration" : 2.0,
        "footStepsToStore" : 4,
        "searchRadius" : 1.0,
        "angleCalculationType" : "USE_CENTER",
        "targetOrientationAngleThreshold" : 45.0
      },
      "idAsTarget" : -1,
      "isChild" : False,
      "isLikelyInjured" : False,
      "mostImportantEvent" : None,
      "salientBehavior" : "TARGET_ORIENTED",
      "groupIds" : [ ],
      "trajectory" : {
        "footSteps" : [ ]
      },
      "groupSizes" : [ ],
      "modelPedestrianMap" : { },
      "type" : "PEDESTRIAN"
    }
    with open(SCENARIO_ROOT + in_file + '.scenario', 'r') as infile:
        data = json.load(infile)
        data['name'] = out_file
        data['scenario']['topography']['dynamicElements'].append(ped)
    with open(SCENARIO_ROOT + out_file + '.scenario', 'w') as outfile:
        json.dump(data, outfile, indent=2)

In [ ]:
addPedestrian('corner', 'corner_modified', 1, 24.0, 2.5, [1], 0.8125422260781998)

In [ ]:
# Matrix with the output of the "real" scenario
def output_matrix():
    output_filename = './output/bottleneck_osm/postvis.trajectories'
    with open(output_filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ')
        next(reader)
        matrix = []
        for row in reader:
            a = [int(row[0]), int(row[1]), float(row[2]), float(row[3]), int(row[4]), float(row[5])]
            matrix.append(a)
        return matrix

In [ ]:
# Creating timestamp files from a template
matrix = output_matrix()
step_max = max([list(i) for i in zip(*matrix)][0])
for i in range(1, step_max + 1):
    with open(BOTTLENECK_GNM, 'r') as infile:
        data = json.load(infile)
    timestamp = 'timestamp_' + str(i)
    new_sec_file = SCENARIO_ROOT + timestamp + '.scenario'
    with open(new_sec_file, 'w') as new_sec:
        json.dump(data, new_sec, indent=2)
    pedestrians = list(filter(lambda x: x[0] == i, matrix))
    for p in pedestrians:
        p_id = p[1]
        p_x = p[2]
        p_y = p[3]
        # TODO What happens with multiple targets?
        p_target = p[4]
        p_speed = p[5]
        addPedestrian(timestamp, timestamp, p_id, p_x, p_y, [p_target], p_speed)